In [1]:
from dotenv import load_dotenv
import os

load_dotenv("/home/dario/Dropbox/ai/key.env",override=True)
os.environ['OPENAI_API_KEY'] = os.environ.get("PNH_RAG_OPENAI_KEY")
os.environ['LLAMA_CLOUD_API_KEY'] = os.environ.get("PNH_RAG_LLAMAPARSE_KEY")
os.environ['PINECONE_API_KEY'] = os.environ.get("PNH_RAG_PINECONE_API_KEY")


In [2]:
import logging
import sys
from IPython.display import display, Markdown

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
pdf_directory = "/home/dario/Dropbox/ai/pnh-qa/pnh_data/pdf"

In [4]:
import nest_asyncio

nest_asyncio.apply()


In [6]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    language="es",
    num_workers=8,
)

file_extractor = {".pdf": parser}

filename_fn = lambda filename: {"file_name": filename}
# automatically sets the metadata of each document according to filename_fn
documents = SimpleDirectoryReader(
    input_dir=pdf_directory, file_extractor=file_extractor, file_metadata=filename_fn, required_exts=[".pdf"]
).load_data()


INFO:httpx:HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"
Started parsing the file under job_id ffe6e8d4-f1b5-4a38-9de5-87014f13fc79
INFO:httpx:HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/ffe6e8d4-f1b5-4a38-9de5-87014f13fc79 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/ffe6e8d4-f1b5-4a38-9de5-87014f13fc79 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/ffe6e8d4-f1b5-4a38-9de5-87014f13fc79 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/ffe6e8d4-f1b5-4a38-9de5-87014f13fc79 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/ffe6e8d4-f1b5-4a38-9de5-87014f13fc79 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/ffe6e8d4-f1b5-4a38-9de5-87014f13fc

In [7]:
print(len(documents))

34


In [8]:
print(documents[0].text[:1000])

# 2012 'Ano de Homenaje al Doctor D. Manuel Belgrano

Ministerio de Turismo Administracion de Parques Nacionales LeyN" 22.351 0 0 0 6 6 3 SAN CARLOS DE BARILOCHE, 05 NOV 2012

VISTO la Disposicion del Senor Intendente del Parque Nacional Nahuel Huapi No 00747/2009, mediante la cual se fijo la normativa que ordenaba las actividades recreativas y/o deportivas de montana a los llamados GRUPOS ORGANIZADOS, modificando anteriores normativas en tal sentido cuyas actuaciones han recaido en el Expediente 1571/2009 (EXP-PNA: 003963/2011) Y;

CONSIDERANDO:

Que si bien la misma al momento de Su elaboracion fue pensada como una herramienta instrumento eficaz necesario que permitiria ordenar encuadrar adecuadamente los distintos grupos participantes Y/0 intervinientes en tal actividad; la experiencia recogida hasta el presente al menos en terminos de aplicacion practica concreta dejo al descubierto una serie de falencias imprevisiones que terminaron complicando dificultando aun mas la situacion qu

In [9]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
# text-embedding-3-small : 1536 dimensions
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [10]:
from llama_index.core.node_parser import MarkdownElementNodeParser

#node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo"), num_workers=8)
node_parser = MarkdownElementNodeParser(num_workers=8)

In [12]:
nodes = node_parser.get_nodes_from_documents(documents=documents)

2it [00:00, 8160.12it/s]
  0%|          | 0/2 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 50%|█████     | 1/2 [00:02<00:02,  2.75s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 2/2 [00:04<00:00,  2.05s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 2206.37it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

...

 92%|█████████▏| 12/13 [00:04<00:00,  3.63it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 13/13 [00:04<00:00,  2.87it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 6078.70it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 1/1 [00:01<00:00,  1.88s/it]


In [13]:
from pinecone import Pinecone, ServerlessSpec
pc_api_key = os.environ["PINECONE_API_KEY"]

pc = Pinecone(api_key=pc_api_key)

In [14]:
# delete if needed
# pc.delete_index("name")

In [15]:
pc.create_index(
    name="pnh-rag",
    dimension=1536,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [16]:
pinecone_index = pc.Index("pnh-rag")

In [17]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex

In [18]:
# initialize without metadata filter
from llama_index.core import StorageContext

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [19]:
# index = VectorStoreIndex(nodes,storage_context=storage_context)
index = VectorStoreIndex(nodes,storage_context=storage_context)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors:   0%|          | 0/533 [00:00<?, ?it/s]

In [20]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()

In [30]:
response = query_engine.query("Qué hay que tener en cuenta para la observación de aves?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [31]:
display(Markdown(f"{response}"))

Es importante tener en cuenta la ubicación y el entorno donde se realiza la observación de aves, respetar las velocidades máximas para evitar perturbar a las aves, registrar los avistajes y observaciones de manera detallada, y difundir la importancia de cuidar y conservar las aves a quienes nos rodean, incluyendo clientes, choferes y colegas. Los guías del área protegida también juegan un papel crucial como "agentes de conservación" al colaborar en la protección de la fauna.